In [4]:
!pip install opencv-python imagehash

In [5]:
import os
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

import cv2
import imagehash
from PIL import Image

In [6]:
def get_frame_histogram(frame):
    """
    Вычисляет гистограмму цвета для кадра.
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Используем гистограмму для каналов H и S
    hist = cv2.calcHist([hsv], [0, 1], None, [50, 60], [0, 180, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()


def are_histograms_similar(hist1, hist2, threshold=0.7):
    """
    Сравнивает две гистограммы с использованием коэффициента корреляции.
    """
    correlation = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return correlation > threshold


def extract_diverse_frames(video_path, max_frames=30):
    """
    Извлекает максимальное количество (не более max_frames) наиболее разнообразных кадров из видео.
    """
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Ошибка: Не удалось открыть видео.")
        return []

    fps = cap.get(cv2.CAP_PROP_FPS)
    selected_frames = []
    histograms = []

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps
    interval = max(1, int(frame_count / (max_frames * 2)))  # Интервалы для выбора кадров

    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Получение текущего номера кадра
        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)

        # Вычисление времени в секундах
        time_seconds = frame_number / fps

        if current_frame % interval == 0:
            hist = get_frame_histogram(frame)

            # Проверяем, похож ли текущий кадр на уже выбранные
            similar = False
            for existing_hist in histograms:
                if are_histograms_similar(hist, existing_hist):
                    similar = True
                    break

            if not similar:
                selected_frames.append({"video_name": video_name, "frame": frame, "time_seconds": time_seconds})
                histograms.append(hist)
                if len(selected_frames) >= max_frames:
                    break

        current_frame += 1

    cap.release()
    return selected_frames


def select_unique_frames(video_path, max_frames=30, hash_size=8, frame_interval=10):
    """
    Извлекает уникальные кадры из видео.

    :param video_path: Путь к видеофайлу.
    :param max_frames: Максимальное количество уникальных кадров.
    :param hash_size: Размер хэша для сравнения (чем больше, тем точнее).
    :param frame_interval: Интервал между кадрами для выборки.
    :return: Список уникальных кадров в формате OpenCV.
    """
    # Получаем имя видео без расширения
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Ошибка открытия видео файла.")
        return []

    fps = cap.get(cv2.CAP_PROP_FPS)

    unique_frames = []
    hashes = set()
    frame_count = 0

    while cap.isOpened() and len(unique_frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Получение текущего номера кадра
        frame_number = cap.get(cv2.CAP_PROP_POS_FRAMES)

        # Вычисление времени в секундах
        time_seconds = frame_number / fps

        if frame_count % frame_interval == 0:
            # Преобразуем кадр из BGR (OpenCV) в RGB (PIL)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame_rgb)

            # Вычисляем хэш кадра
            frame_hash = imagehash.average_hash(pil_image, hash_size=hash_size)

            if frame_hash not in hashes:
                hashes.add(frame_hash)
                unique_frames.append({"video_name": video_name, "frame": frame, "time_seconds": time_seconds})

        frame_count += 1

    cap.release()
    return unique_frames


def save_frames(frames, output_folder):
    """
    Сохраняет список кадров в указанную папку.

    :param frames: Список кадров в виде массивов NumPy
    :param output_folder: Путь к папке для сохранения изображений
    """
    import os
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for idx, frame in enumerate(frames):
        frame_filename = f"{frame['video_name']}_{frame['time_seconds']}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)
        cv2.imwrite(frame_path, frame['frame'])


def extract_frames(video_path, output_dir):
    unique_frames = extract_diverse_frames(video_path)
    save_frames(unique_frames, output_dir)
    print(f"Извлечено {len(unique_frames)} кадров из {video_path}")


def process_video(video_file, input_dir, output_dir):
    video_path = os.path.join(input_dir, video_file)
    extract_frames(video_path, output_dir)
    return video_file


def process_videos(input_dir, output_dir, max_workers=10):
    # Проверяем, существует ли выходная директория, если нет - создаем
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Получаем список файлов в директории
    files = os.listdir(input_dir)

    # Фильтруем только файлы с расширением .webm
    video_files = [f for f in files if f.lower().endswith('.mp4')]

    count = 0
    total = len(video_files)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Создаем частичную функцию с фиксированными параметрами input_dir и output_dir
        future_to_video = {executor.submit(process_video, vf, input_dir, output_dir): vf for vf in video_files}

        for future in as_completed(future_to_video):
            video_file = future_to_video[future]
            try:
                result = future.result()
                count += 1
                print(f"Обработано видео {result} {count}/{total}")
            except Exception as e:
                print(f"Ошибка при обработке видео {video_file}: {e}")


def remove_dir(path):
    try:
        shutil.rmtree(path)
        print(f"Директория '{path}' и все её содержимое успешно удалены.")
    except Exception as e:
        print(f"Ошибка при удалении директории: {e}")


if __name__ == "__main__":
    input_directory = '../test_data_yappy/test_dataset'
    output_directory = '../dataset_new_test/'
    remove_dir(output_directory)
    os.mkdir(output_directory)

    # Запускаем обработку видео
    process_videos(input_directory, output_directory)


Ошибка при удалении директории: [Errno 2] No such file or directory: '../dataset_new_test/'
Извлечено 11 кадров из ../test_data_yappy/test_dataset/09e4378c-f7a6-4345-9925-e4460a2215a3.mp4
Обработано видео 09e4378c-f7a6-4345-9925-e4460a2215a3.mp4 1/1000
Извлечено 21 кадров из ../test_data_yappy/test_dataset/2b7a2406-32af-44de-9168-9fcfaaa4457b.mp4
Обработано видео 2b7a2406-32af-44de-9168-9fcfaaa4457b.mp4 2/1000
Извлечено 9 кадров из ../test_data_yappy/test_dataset/3f63b5c2-1c4b-414b-b0db-ec022ea003cf.mp4
Обработано видео 3f63b5c2-1c4b-414b-b0db-ec022ea003cf.mp4 3/1000
Извлечено 6 кадров из ../test_data_yappy/test_dataset/4fbbfca7-70c1-44dd-9ff8-71d4fb98df69.mp4
Обработано видео 4fbbfca7-70c1-44dd-9ff8-71d4fb98df69.mp4 4/1000
Извлечено 14 кадров из ../test_data_yappy/test_dataset/31edad77-95fa-4d70-94e2-25808157556d.mp4
Обработано видео 31edad77-95fa-4d70-94e2-25808157556d.mp4 5/1000
Извлечено 11 кадров из ../test_data_yappy/test_dataset/0bda6968-eec4-4757-9ebc-52ba3d3aefd4.mp4
Обработан